# Ticker client

A notebook demonstrating asynchronous algorithms to maintain a "local" copy of the ticker data from the server.

We detail algorithms for two different levels of hypothetical REST API support:
- the exchange supports information on all tickers being downloaded in one request.
- the exchange supports only a single ticker's information being downloaded in one request.

Due to the ubiquity of REST API rate limitations, in the latter case it is usually infeasible to develop real-time information on all markets at once using the REST API alone.

In both cases, the REST and websocket APIs can be combined to maintain real-time information on all markets.

In [15]:
import aiohttp
import ast
import asyncio
import coinblockpro
import copy
import nest_asyncio
import time
import websockets

nest_asyncio.apply()  # lets asyncio be run in Jupyter inside Tornado's event loop

### Full-ticker

The full-ticker algorithm first establishes the websocket client, and builds a buffer of the markets it has acuired information for, keeping only the latest.

After a time-delay, the REST API is contacted once to retrieve the ticker information. The buffer of websocket data is used to update the ticker information from the REST API.

From this point on, so long as the websocket is maintained, the market information will be up-to-date.

In [26]:
class FullTicker:
    """Class to monitor an exchange's markets where the exchange
    offers the ability to download info on all tickers at once over
    the REST API.
    """
    
    def __init__(self):
        self.ticker_data = {}  # market: price
        self.rest_data = None

    async def get_full_ticker(self):
        """Coroutine to fetch the full ticker over the REST API.
        Sleep at beginning to give the websocket time to establish.
        """       
        await asyncio.sleep(3)
        url = 'http://0.0.0.0:8000/full_ticker'
        
        async with aiohttp.ClientSession() as session:
            async with session.get(url) as response:
                payload = await response.json()
                if not response.status == 200:
                    raise ValueError('Request failed!')
                if not 'result' in payload:
                    raise ValueError('Request error.')
                self.rest_data = payload['result']
                self.insert_rest_values(rest_data)
            
    def insert_rest_values(self, data):
        """Blocking function to insert the rest API data into the 
        ticker_data container."""
        print('assigning rest data')
        temp_dict = copy.deepcopy(self.ticker_data)
        self.ticker_data = data
        for k, v in temp_dict.items():
            self.ticker_data[k] = v
        print('rest data assigned')
            
    async def subscribe(self, period):
        """Coroutine to subscribe to the websocket ticker feed for 
        the given period (in seconds)."""
        print('establishing websocket feed')
        time_now = time.time()
        async with websockets.connect('ws://0.0.0.0:8001/ticker_feed') as ws:
            while time.time() < time_now + period:
                resp = await ws.recv()
                my_dict = ast.literal_eval(resp)
                for k, v in my_dict.items():
                    print(f'Updating market {k} to price {v}')
                    self.ticker_data[k] = v
                
    async def monitor_markets(self):
        """Monitor markets using the REST and websocket APIs."""
        await asyncio.gather(
            self.get_full_ticker(),
            self.subscribe(30)
        )

In [27]:
f = FullTicker()
asyncio.run(f.monitor_markets())

establishing websocket feed
Updating market dot_eur to price 2299.946
Updating market btc_usd to price 8761.274
assigning rest data
rest data assigned
Updating market btc_dot to price 479.393
Updating market bch_eur to price 5299.653
Updating market eur_usd to price 10908.104
Updating market xlm_usd to price 1570.764
Updating market btc_eth to price 2677.865
Updating market btc_xrp to price 9774.694


In [30]:
ticker_data['btc_usd']

8761.274